In [6]:
import os
import json
import requests
from datetime import datetime

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool
#from langchain.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate



In [9]:
# =========================
# FULL AGENTIC AI SYSTEM
# =========================

import os
import json
import requests
from datetime import datetime

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool


# =========================
# TOOLS
# =========================

@tool
def perceive_weather(city: str) -> dict:
    """Perceive: Fetch live weather data"""
    api_key = os.getenv("OPENWEATHER_API_KEY")
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    data = requests.get(url).json()

    return {
        "city": city,
        "temperature": data["main"]["temp"],
        "weather": data["weather"][0]["description"],
        "wind_speed": data["wind"]["speed"],
        "timestamp": datetime.utcnow().isoformat()
    }


@tool
def analyze_risk(weather: dict) -> dict:
    """Analyze: Convert raw weather into delivery risk"""
    risk = "LOW"
    if "rain" in weather["weather"].lower() or weather["wind_speed"] > 10:
        risk = "HIGH"

    return {
        "city": weather["city"],
        "risk_level": risk,
        "reason": f"{weather['weather']} | wind={weather['wind_speed']} m/s"
    }


@tool
def update_delivery(city: str, decision: str):
    """Act: Update real delivery system state"""
    file = "delivery_status.json"

    data = json.load(open(file)) if os.path.exists(file) else {}
    data[city] = {
        "decision": decision,
        "updated_at": datetime.utcnow().isoformat()
    }

    json.dump(data, open(file, "w"), indent=2)
    return f"Delivery status updated for {city}: {decision}"


@tool
def learn(event: dict):
    """Learn: Persist agent experience"""
    file = "learning_log.json"

    history = json.load(open(file)) if os.path.exists(file) else []
    history.append(event)

    json.dump(history, open(file, "w"), indent=2)
    return "Learning recorded successfully"


# =========================
# LLM
# =========================

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)


# =========================
# CREATE AGENT (NO PROMPT!)
# =========================

tools = [
    perceive_weather,
    analyze_risk,
    update_delivery,
    learn
]

agent = create_react_agent(
    model=llm,
    tools=tools
)


# =========================
# RUN AGENT
# =========================

agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Assess today's delivery feasibility for Bangalore and take appropriate action"
        }
    ]
})


/var/folders/m8/lr5c6v793qbcszm54dsf6y440000gn/T/ipykernel_96857/2472240049.py:97: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


KeyError: 'main'

In [8]:
# =========================
# FULL AGENTIC AI SYSTEM
# =========================

import os
import json
import requests
from datetime import datetime

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool
from langchain_core.prompts import PromptTemplate


# =========================
# TOOLS
# =========================

@tool
def perceive_weather(city: str) -> dict:
    """Perceive: Fetch live weather data"""
    api_key = os.getenv("OPENWEATHER_API_KEY")
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    data = requests.get(url).json()

    return {
        "city": city,
        "temperature": data["main"]["temp"],
        "weather": data["weather"][0]["description"],
        "wind_speed": data["wind"]["speed"],
        "timestamp": datetime.utcnow().isoformat()
    }


@tool
def analyze_risk(weather: dict) -> dict:
    """Analyze: Convert raw weather into delivery risk"""
    risk = "LOW"
    if "rain" in weather["weather"].lower() or weather["wind_speed"] > 10:
        risk = "HIGH"

    return {
        "city": weather["city"],
        "risk_level": risk,
        "reason": f"{weather['weather']} | wind={weather['wind_speed']} m/s"
    }


@tool
def update_delivery(city: str, decision: str):
    """Act: Update real delivery system state"""
    file = "delivery_status.json"

    data = json.load(open(file)) if os.path.exists(file) else {}
    data[city] = {
        "decision": decision,
        "updated_at": datetime.utcnow().isoformat()
    }

    json.dump(data, open(file, "w"), indent=2)
    return f"Delivery status updated for {city}: {decision}"


@tool
def learn(event: dict):
    """Learn: Persist agent experience"""
    file = "learning_log.json"

    history = json.load(open(file)) if os.path.exists(file) else []
    history.append(event)

    json.dump(history, open(file, "w"), indent=2)
    return "Learning recorded successfully"


# =========================
# LLM + PROMPT
# =========================

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

prompt = PromptTemplate.from_template("""
You are a logistics operations agent.

You MUST follow this loop:
Perceive → Analyze → Reason → Plan → Act → Learn

Your responsibilities:
1. Perceive live weather conditions
2. Analyze delivery risk
3. Decide whether delivery should proceed or be delayed
4. Update the delivery system state
5. Record learning for future improvement

Task: {input}
{agent_scratchpad}
""")


# =========================
# CREATE AGENT (FIXED)
# =========================

tools = [
    perceive_weather,
    analyze_risk,
    update_delivery,
    learn
]

agent = create_react_agent(
    model=llm,     # ✅ THIS IS THE FIX
    tools=tools,
    prompt=prompt
)


# =========================
# RUN AGENT
# =========================

agent.invoke({
    "input": "Assess today's delivery feasibility for Bangalore and take appropriate action"
})


/var/folders/m8/lr5c6v793qbcszm54dsf6y440000gn/T/ipykernel_96857/3211591357.py:115: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


KeyError: "Input to PromptTemplate is missing variables {'agent_scratchpad', 'input'}.  Expected: ['agent_scratchpad', 'input'] Received: ['messages', 'remaining_steps']\nNote: if you intended {agent_scratchpad} to be part of the string and not a variable, please escape it with double curly braces like: '{{agent_scratchpad}}'.\nFor troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/INVALID_PROMPT_INPUT "

In [ ]:
@tool
def perceive_weather(city: str) -> dict:
    """
    Fetch live weather data for a city.
    """
    api_key = os.getenv("OPENWEATHER_API_KEY")
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    
    response = requests.get(url).json()
    
    return {
        "temperature": response["main"]["temp"],
        "weather": response["weather"][0]["description"],
        "wind_speed": response["wind"]["speed"]
    }


In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)


In [ ]:
agent_prompt = PromptTemplate.from_template("""
You are a logistics decision-making agent.

Your task:
1. Understand weather conditions
2. Assess delivery risk
3. Decide whether action is needed
4. Plan steps to mitigate delivery issues
5. Take action using available tools

Think step by step.

Question: {input}
{agent_scratchpad}
""")


In [ ]:
@tool
def execute_delivery_action(decision: str):
    """
    Execute delivery-related action.
    """
    return f"Action executed: {decision}"


In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)


In [ ]:
tools = [
    perceive_weather,
    analyze_weather,
    execute_delivery_action
]

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=agent_prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

agent_executor.invoke({
    "input": "Check today's delivery risk for Bangalore and take appropriate action"
})
